# Census County GIS Data Cross-Year Exploration

This Python (ArcPy) script arose from the data exploration phase of an ArcGIS script + Dashboard project begun in summer 2025 (at the beginning of my post-[Deferred Resignation Program](https://en.wikipedia.org/wiki/2025_U.S._federal_deferred_resignation_program) professional gap year[ish]).

The project centers on an ArcGIS Dashboard that displays FEMA US county Disaster Declarations, and is powered by a script that automatically updates the map on a daily basis. This Dashboard promises to be a near-real-time, interactive version of static PDFs I often made for USDA HSD (Homeland Security Division) personnel with whom I worked while I was with USDA GEO (which is the role I exited via [DRP](https://en.wikipedia.org/wiki/2025_U.S._federal_deferred_resignation_program)).

This Dashboard has at its core a simple dataset comprising US county geographies (which are joined to FEMA Disaster Declaration data via FIPS codes). I know from my work with USDA—specifically from my partnership on multiple projects with FSA—that FIPS codes do occasionally change from year to year. (An unusual and somewhat extreme example is the [shake-up of Connecticut counties in 2022](https://www.federalregister.gov/documents/2022/06/06/2022-12063/change-to-county-equivalents-in-the-state-of-connecticut)).

If I want to represent FEMA Disaster Declaration data across multiple years, I must ensure that my US county data reflects all FIPS codes from the same span of years. If, for example, I simply used the most recent US Counties dataset from US Census Bureau, any Disaster Declarations from before 2022 for Connecticut (specifically, County FIPS codes from before 2022) would not join to post-2022 counties, and those Declarations would therefore not be reflected in my Dashboard.

This Python script therefore represented an interest on my part in discovering just how many FIPS codes have changed within my temporal range of data interest.

I also know from my time with USDA that US County geographies are also subject to occasional change. Often these changes are small, e.g. an adjustment along a migrating meandering river. Nevertheless I figure I may as well explore the US county geography changes as long as I am already programmatically mucking about in the guts of US county geodatabases.

Additional ulterior motives: I don't really have an updated (Python 3.x) script in my GIS Portfolio which primary purpose is mucking about in the guts of geodatabases: data access cursors and all that. And, finally: Python is just obscenely fun.

P.S. This script took two days to put together. The ONLY thing ChatGPT helped with was clueing me into the existence of the "template" parameter for the ArcPy .CreateFeatureclass() method.

### Imports

In [1]:
import os
import sys
from zipfile import ZipFile
import arcpy

### Switches

In [2]:
# Switch to toggle between running on US Census Counties or US Census Tribal Areas
# (since Disaster Declarations can be for either)

# Input "Counties" or "Tribal":
dataset = "tribal"

# Switch to toggle between comparing based solely on GEOID
# or both GEOID and feature geometry:

# Input "geoid" or "both":
compare = "geoid"

# A value (a very small value) representing the number of degrees
# that two counties or tribal areas with the same GEOID may differ in SHAPE@LENGTH
# above which we will consider their geometries sufficiently different that
# both geometries are written to the output feature class.

# How much is significant? I ran the input counties through the wringer multiple times
# with different values; the smaller the difference, the more counties will be written.
# For example, running Census years 2013-2025,
# the numbers are as follows:
# .01: 3675 counties
# .005: 3945 counties | .001: 4897 counties
# .0005: 5454 counties | .0001: 6877 counties
# .00001: 8646 counties
# Obviously the numbers indicate that over the years many, many, tiny changes
# and adjustments are made (happen? is it deliberate? magic?) to US Census counties.

degrees = .0001 # Value is very SMALL because unit is DEGREES

### Variables

In [3]:
# The base directory in which my zipped Census files, output GDB etc reside.
base_dir = r"C:\Users\Misti\OneDrive\PROJECTS\2025_08_Multi-Year_Census_Counties"

# Output geodatabase, to which my Frankenstein multi-year US counties feature class will be written
output_gdb = r"2025_08_Multi-Year_Census_Counties.gdb"

# Set variables based on whether switch is "Counties" or "Tribal":
if dataset.lower() == "counties":
    # The name of the script's single output feature class
    output_fc = "multi_year_census_counties"

    # The name of the input feature class from US Census Zips
    input_fc = "County"

    # Names of the folders in which the US Census Bureau's
    # zipped and unzipped files reside (files are unzipped programmatically)
    zip_dir_name = r"Zipped_Counties"
    unzip_dir_name = r"Unzipped_Counties"

elif dataset.lower() == "tribal":
    # The name of the script's single output feature class
    output_fc = "multi_year_native_american_areas"

    # The name of the input feature class from US Census Zips
    input_fc = "AmericanIndian_AlaskaNative_NativeHawaiianArea"

    # Names of the folders in which the US Census Bureau's
    # zipped and unzipped files reside (files are unzipped programmatically)
    zip_dir_name = r"Zipped_Tribal"
    unzip_dir_name = r"Unzipped_Tribal"

else:
    print("Please set a valid selector for input dataset!" \
          "Qitting script now.")
    sys.exit()

# Creation of full paths to output GDB and feature class
full_output_gdb = os.path.join(base_dir, output_gdb)
full_output_fc = os.path.join(full_output_gdb, output_fc)


### Extract All Zips
This function looks at the folder containing the freshly-downloaded ZIP files Census Bureau and unzips the contents of all (a single GDB for each) to the specified output folder.

In [4]:
def extract_all_zips():

    # Create the full path to the folder containing all the zips
    for file in os.listdir(os.path.join(base_dir, zip_dir_name)):

        # Over-cautious bit to ignore any non-zip files
        # that should theoretically not be in this folder
        if not file.endswith(".zip"):
            continue
        
        # Do the actual unzipping
        with ZipFile(os.path.join(base_dir, zip_dir_name, file), 'r') as zip_obj:
            zip_obj.extractall(os.path.join(base_dir, unzip_dir_name))

# This function generally wouldn't need to be called more than once,
# even during development/troubleshooting, so leaving it commented out
# in case of an errant click on "run all cells above" etc.
# UNCOMMENT THE LINE BELOW TO ACTUALLY CALL THIS FUNCTION AND EXTRACT ZIPS
#extract_all_zips()

### Get Counties

This function checks to make sure each unzipped GDB actually contains a feature class called "County". (Ideally safe to assume, but I've worked in fedland too long to make assumptions about tidy data) It rounds up the full paths of all the feature classes it finds into a list "found_fcs".

In [5]:
#Initialize empty list to hold full county FC paths
found_fcs = []

def get_counties():

    # Keeping track of whether any GDBs don't have a "County" FC
    unfound = 0
    unfound_gdbs = []

    # Loop through the GDBs
    for file in os.listdir(os.path.join(base_dir, unzip_dir_name)):

        # Ignore any non-GDBs that happen to have gotten into this dir
        if not file.endswith(".gdb"):
            continue

        # For each GDB, create full path and set arcpy env
        gdb = os.path.join(base_dir, unzip_dir_name, file)
        arcpy.env.workspace = gdb
        
        # Check for "County" FC; if found, plop it in my list
        if not input_fc in arcpy.ListFeatureClasses():
            unfound +=1
            unfound_gdbs.append(gdb)
        else:
            found_fcs.append(os.path.join(base_dir, unzip_dir_name, file, input_fc))

    # Reporting on any missing "County" FCs
    if unfound:
        print(f"Heads up! Didn't find a {input_fc} FC in these GDBs: {unfound_gdbs}")
    else:
        print(f"Yay! Found a {input_fc} FC in all gdbs!")

    return found_fcs

found_fcs = get_counties()


Yay! Found a AmericanIndian_AlaskaNative_NativeHawaiianArea FC in all gdbs!


### Create Year: Full Path Dictionary

I'm interested not only in how many variations of a given county boundary exist, but in which came from what year. Obviously, because a single US county feature class from Census Bureau represents a single year, that data is not in the fc attribute table itself. I need to add the year as a field to my final multi-year output county feature class. In order to add it, I must first track it. I do so by converting my Python list of full path strings (which technically contain the year) to a Python dictionary in which the year is key and the full path is the value.

In [6]:
# Initialize dict that will hold year:fullpath key:value pairs
year_fcs_dict = {}

def create_dict(year_fcs_dict):

    # Resetting intermediate and final lists/dicts
    # While testing/developing
    pre_dict = {}
    pre_list = []
    year_fcs_dict.clear()

    # Loop through my list of feature class full paths
    for fc in found_fcs:

        # Isolate the year by splitting the full path string
        year = fc.split("tlgdb_")[1].split("_a_")[0]

        # Chuck the year in the dict and assign the full path as its value
        pre_dict[year] = fc

    # I think it's tidier and more logical to process the output dictionary
    # in reverse chronological order. That way, if a given county FIPS 
    # and associated geometry exist across multiple years (i.e. the geometry has not changed),
    # the year associated with that row in my final output feature class
    # should reflect the most recent year in which that FIPS/geometry pair is found.
    # To ensure this is what happens, I reverse-sort my dict here.
    pre_list = sorted(pre_dict.items(), reverse=True)
    year_fcs_dict = dict(pre_list)

    return year_fcs_dict

year_fcs_dict = create_dict(year_fcs_dict)

### Create Output Geodatabase and Feature Class

Pretty self-explanatory

In [7]:
def create_output():

    # I was initially just doing year_fcs_dict[0] here,
    # but when my insert cursor failed, I realized that a field was added
    # to the annual FC sometime between my first and last GDBs.
    # I figure it's more likely that Census Bureau will add fields 
    # in the future rather than remove them, so I'm using as my template
    # feature class the NEWEST feature class in my dict.
    template_fc = year_fcs_dict[max(year_fcs_dict.keys())]

    # Get the spatial reference for my template
    spatial_ref = arcpy.Describe(template_fc).spatialReference

    # Check whether my output GDB already exists; if not, make it
    if not arcpy.Exists(full_output_gdb):
        print("Creating output GDB...")
        arcpy.management.CreateFileGDB(base_dir, output_gdb)
    else:
        print("Yay! Output GDB already exists.")

    # Check whether my output feature class already exists; if not, make it
    if not arcpy.Exists(full_output_fc):
        print("Creating output feature class...")
        arcpy.management.CreateFeatureclass(full_output_gdb, output_fc, "POLYGON", template=template_fc, spatial_reference=spatial_ref)
    else:
        print("Yay! Output feature class already exists.")

    return full_output_fc

create_output()

Yay! Output GDB already exists.
Yay! Output feature class already exists.


'C:\\Users\\Misti\\OneDrive\\PROJECTS\\2025_08_Multi-Year_Census_Counties\\2025_08_Multi-Year_Census_Counties.gdb\\multi_year_native_american_areas'

### Add Year Field

As mentioned above, I want to track in the output feature class which row came from which year. To do so, I am adding a "Census_Year" field to the output feature class.

In [8]:
def add_year_field():
    
    # I wouldn't technically have to do this bit because
    # if the field already exists, arcpy will just do nothing and move on...
    #...but hey, more messaging is better than less...?
    if not "Census_Year" in (f.name for f in arcpy.ListFields(full_output_fc)):
        print("Adding 'Census_Year' field")
        arcpy.management.AddField(full_output_fc, "Census_Year", "SHORT", field_length=4, field_alias="Census Year")

    else:
        print("Field 'Census_Year' alread exists! Moving on...")

add_year_field()

Field 'Census_Year' alread exists! Moving on...


### Insert Counties

Here is the meat-and-potatoes function that actually iterates all input feature classes and writes the appropriate rows to the output feature class.

In [11]:
def insert_counties(year_fcs_dict):

    # Iterate through my dictionary of year:full fc path pairs
    for year, fc in year_fcs_dict.items():

        # A list of lists may have been more...structurally logical here,
        # but let's face it, a one-liner dictionary cursor comprehension is just sexy.
        # I only needed the pair of GEOID and SHAPE@LENGTH values for each input row,
        # to compare year-over year. Dictionary technically works.
        gid_shape_dict = {row[0]: row[1] for row in arcpy.da.SearchCursor(full_output_fc, ["GEOID", "SHAPE@LENGTH"])}

        # Generate the list of fields from the input feature class (remember
        # it's from the most recent year) for use with the input fc search cursor.
        fields = [f.name for f in arcpy.ListFields(fc)]

        # Add the SHAPE@ token because I won't get very far
        # writing new geometries without it, and add SHAPE@LENGTH
        # for a more efficient geometry comparison year-over-year.
        fields.append("SHAPE@")
        fields.append("SHAPE@LENGTH")

        # Make a copy of fields for use with the insert cursor.
        # "Census_Year" field only exists in the output fc, not the inputs,
        # So to make both cursors happy there must be two field lists.
        # Also creating the insert cursor field list from the search cursor
        # field list ensures consistent field ordering.
        ifields = fields.copy()
        ifields.append("Census_Year")

        # Fire up the search cursor to iterate through the current input fc
        with arcpy.da.SearchCursor(fc, fields) as scursor:
            for row in scursor:

                # Cut down on verbose duplicated syntax in the logic below
                gid = row[fields.index("GEOID")]
                shape = row[fields.index("SHAPE@LENGTH")]
                
                # Now for the (ok really not so) fancy footwork:
                # IF the GEOID (i.e. FIPS code) is already in the output feature class...
                if gid in gid_shape_dict:

                    # If comparing both geoid AND geometry...
                    if compare == "both":

                        # ...we really only want to write this FIPS dup if the geometries differ significantly.
                        if abs(shape - gid_shape_dict[gid]) < degrees:

                            # If the GEOID is already in the output AND the geometry difference
                            # is BELOW the threshold set in the above line, do not pass go,
                            # do not collect $200, move on to the next row without writing anything.
                            continue

                    # Otherwise if we are only comparing geoid, we can just skip straight to the next row
                    # Nothin' to see here, folks
                    elif compare == "geoid":
                        continue

                # Otherwise, fire up the insert cursor...
                with arcpy.da.InsertCursor(full_output_fc, ifields) as icursor:
                    
                    # ...don't forget to tack the source year onto the end of the row...
                    new_row = row + (year,)

                    # And actually do the thing
                    icursor.insertRow(new_row)

insert_counties(year_fcs_dict)

### Screenshot of output feature class
![CONUS US output feature class](https://raw.githubusercontent.com/AlderMaps/arcgis-arcpy/refs/heads/main/multi-year_census_counties.png "Output Feature Class (with geometry difference at .0001 degrees)")75% transparency applied to polygons; brighter counties == more (slightly incongruent) geometries written for a given GEOID.<br>
I.e. brighter counties/areas have had more geometry changes over the years.

In [10]:
# FOR USE WHILE BUILDING SCRIPT AND TROUBLESHOOTING ONLY!!!

def truncate_table():
    
    # Clear out all rows in the output feature class (for dev/testing)
    arcpy.management.TruncateTable(full_output_fc)

truncate_table()


### Update Aug 12, 2025

Put the finishing touches on this and one hour later realize that US Census *actually has* [rest services](https://tigerweb.geo.census.gov/arcgis/rest/services). 🤯

Slightly mortified that I didn't realize this before today 😳; however the 1999 vibe [their page](https://tigerweb.geo.census.gov/tigerwebmain/TIGERweb_restmapservice.html) gives off makes me suspect I'm not the only one who didn't know, and that the page doesn't get a lot of traffic (or love). Ah well, still good to have a script in my portfolio demonstrating working with zip files. 😅

To-do: Re-do this script leveraging services. 🤣